# Guidelines for Prompting

In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

First, if you are running on HPC, you need to initiate the gcloud SDK using `gcloud auth application-default login`. Once done, restart the kernel.

In [1]:
import vertexai
from vertexai.language_models import TextGenerationModel

GCP_PROJ_ID = "xxxxx" # replace with your GCP project ID
vertexai.init(project=GCP_PROJ_ID, location="us-central1")

In [2]:
parameters = {
    "temperature": 0.4,
    "max_output_tokens": 1024,
    "top_p": 0.8,
    "top_k": 25
}
model = TextGenerationModel.from_pretrained("text-bison@001")

Now the format to access PaLM API to get prediction is:
    
```python
response = model.predict("<prompt>", **parameters)
```

## Prompting Principles
* Principle 1: Write clear and specific instructions
* Principle 2: Give the model time to “think”

### Principle 1: Write clear and specific instructions

__Tactic 1: Use delimiters to clearly indicate distinct parts of the input__

> Delimiters can be anything like: ```, """, < >, <tag> </tag>, :

In [3]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.

```{text}```
"""

response = model.predict(prompt, **parameters)
print(response)

You should provide clear and specific instructions to guide the model towards the desired output.


__Tactic 2: Ask for a structured output__

> JSON, HTML

In [4]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""

response = model.predict(prompt, **parameters)
print(response)

```json
[
  {
    "book_id": 1,
    "title": "The Great Gatsby",
    "author": "F. Scott Fitzgerald",
    "genre": "Novel"
  },
  {
    "book_id": 2,
    "title": "Harry Potter and the Sorcerer's Stone",
    "author": "J.K. Rowling",
    "genre": "Fantasy"
  },
  {
    "book_id": 3,
    "title": "The Lord of the Rings",
    "author": "J.R.R. Tolkien",
    "genre": "Fantasy"
  }
]
```


__Tactic 3: Ask the model to check whether conditions are satisfied__

In [5]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = model.predict(prompt, **parameters)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Put a tea bag in a cup.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste.


In [6]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"


\"\"\"{text_2}\"\"\"
"""

response = model.predict(prompt, **parameters)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


__Tactic 4: "Few-shot" prompting__

In [7]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

response = model.predict(prompt, **parameters)
print(response)


 <grandparent>: When the wind blows, the reed bends, but does not break.


### Principle 2: Give the model time to “think”

__Tactic 1: Specify the steps required to complete a task__

In [9]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""

prompt = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""

response = model.predict(prompt, **parameters)
print(response)

__Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion__

In [10]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.

To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not. 

Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```

Student's solution:
```
student's solution here
```

Actual solution:
```
steps to work out the solution and your solution here
```

Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```

Student grade:
```
correct or incorrect
```


Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 

Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
```

Actual solution:
"""

response = model.predict(prompt, **parameters)
print(response)

```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
```

Is the student's solution the same as actual solution just calculated:
```
yes
```

Student grade:
```
correct
```


Likewise, you can create a __Chain of Thought Reasoning__ using Steps and delimiter like `####` and get factual answers. Check out the following template.

```python
delimter = "####"

system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags, \
i.e. {delimiter}.

Step 1:{delimiter} ... Check

Step 2:{delimiter} ... List of available products:
Product 1: ...
Product 2: ...
...
Product N: ...

Step 3:{delimiter} ... Compare query with product details

Step 4:{delimiter} ... Check if true

Step 5:{delimiter}: 
Answer the customer in a friendly tone.
Use the following format:

Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter}

Make sure to include {delimiter} to separate every step.
"""

user_message = "what is the camera quality of pixel 7 compared to pixel 6?"
```

### Iterative Prompt Development

Generate a marketing product description from a product fact sheet

In [11]:
fact_sheet_chair = """
OVERVIEW

- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION

- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS

- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS

- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS

SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.

SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a detailed product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""

response = model.predict(prompt, **parameters)
print(response)

The SWC-100 is a mid-century inspired office chair that is part of a beautiful family of office furniture, including filing cabinets, desks, bookcases, meeting tables, and more. The chair is available in a variety of colors and finishes to suit your needs. It is also available with or without armrests. The SWC-100 is a great choice for both home and business use.

The SWC-100 features a five-wheel plastic coated aluminum base with a pneumatic chair adjust for easy raise/lower action. The chair is also available with soft or hard-floor caster options.

The SWC-100 is made of high-quality materials, including cast aluminum with modified nylon PA6/PA66 coating and HD36 foam. The chair is also made in Italy, which is known for its high-quality craftsmanship.

The SWC-100 is a great choice for anyone looking for a stylish and comfortable office chair. It is available in a variety of colors and finishes to suit your needs, and it is also available with or without armrests. The SWC-100 is a g

__Issue 1: The text is too long.__ Let's limit the number of words/sentences/characters.

In [12]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a detailed product description under 100 words based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""

response = model.predict(prompt, **parameters)
print(response)

The SWC-100 mid-century modern office chair is a stylish and comfortable option for any home or office. It features a sleek, minimalist design with a choice of fabric or leather upholstery and a variety of base finishes. The chair is also available with or without armrests. The SWC-100 is made from high-quality materials and construction, and it is backed by a 5-year warranty.


__Issue 2. Text focuses on the wrong details.__ Ask it to focus on the aspects that are relevant to the intended audience.

In [13]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description under 200 words based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Technical specifications: ```{fact_sheet_chair}```
"""

response = model.predict(prompt, **parameters)
print(response)

The SWC chair is a modern office chair with a sleek design and a variety of features. The chair is made of high-quality materials, including a cast aluminum shell with a modified nylon PA6/PA66 coating and HD36 foam seat. The chair is available in a variety of colors and finishes to suit your needs.


### Model Limitations: Hallucinations

> Boie is a real company, the product name is not real.

In [14]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""

response = model.predict(prompt, **parameters)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a sleek, electric toothbrush that features a built-in accelerometer and gyroscope to track your brushing habits and provide real-time feedback. The toothbrush also has a built-in timer to help you brush for the recommended two minutes, and a pressure sensor to prevent you from brushing too hard. The AeroGlide UltraSlim Smart Toothbrush is available in a variety of colors, and it comes with a charging stand.
